<a href="https://colab.research.google.com/github/matinmazid/York_ML_certificate/blob/master/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load all the dependant libraries

Scenario 1
Load pdf document loader saving to Chromadb

In [3]:
!pip install langchain langchain_community pypdf langchain_core langchain-text-splitters langchain-chroma
!pip install chromadb

In [4]:
apiKey=input("Enter your openai api key")

Enter your openai api keysk-proj-6pNBed7JczaaawxOd9sM_g4I-nJYQH_uD4_m8YV8KNjxJ2IHpeixH9k1HbBgvVtPp09bOmpYcsT3BlbkFJvAa-ef231UKLuJujbjd7xAXuo9Q2ZtfsvMMrryeVB_4oczvSO-grq_xx5Bz6hxpGPyHNqUDY4A


In [5]:
import os
os.environ['OPENAI_API_KEY']=apiKey

Create the lang chain

In [29]:
from langchain.chat_models import ChatOpenAI
import langchain_core
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
model = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo")
output_parser = StrOutputParser()
chatPrompt=ChatPromptTemplate.from_messages(
    [("system","you are a financial specialist with experience in Aritifical Intelligence"),
     ("user","what is XBRL and its relationship to AICPA")]
    )

In [32]:
chain= chatPrompt | model | output_parser

In [33]:
chain.invoke({})


'XBRL stands for eXtensible Business Reporting Language, which is a standardized format for financial reporting. It allows companies to tag financial data in their reports so that it can be easily analyzed and compared across different organizations. XBRL helps to streamline the process of financial reporting and makes it easier for investors, analysts, and regulators to access and analyze financial information.\n\nThe American Institute of Certified Public Accountants (AICPA) is a professional organization for accountants in the United States. The AICPA has been involved in promoting the use of XBRL for financial reporting. They have developed guidelines and resources to help accountants and companies implement XBRL effectively in their financial reporting processes.\n\nIn summary, XBRL is a standardized format for financial reporting that is supported by organizations like the AICPA to improve the efficiency and accuracy of financial reporting.'

In [9]:
!pip install --upgrade --quiet pypdf
!pip install --upgrade --quiet langchain_openai
!pip install tiktoken

https://www.stern.nyu.edu/sites/default/files/assets/documents/ChenChoDouLev2021WP.pdf

https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/

In [37]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import tiktoken
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

loader = PyPDFLoader("ChenChoDouLev2021WP.pdf")
docs = loader.load()


# split and chuck the input data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
# text splitter knows how to extract the text of a sourced doc
all_splits = text_splitter.split_documents(docs)

# all_texts = [cleanStr(doc.page_content) for doc in all_splits]
# all_texts


vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("what is XBRL and its relationship to AICPA")




/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'XBRL stands for eXtensible Business Reporting Language, which is a standard for the electronic communication of business and financial data. The AICPA (American Institute of Certified Public Accountants) is involved in the development and implementation of XBRL standards to improve financial reporting quality and efficiency. The AICPA works with XBRL U.S. and other organizations to set guidance and validation rules for XBRL filings.'